In [2]:
import numpy as np
import gurobipy as grb
import csv
import time
import statsmodels.api as sm
from joblib import Parallel,delayed

def five_variate(n,SNR,rho):
    beta1 = np.array([1,1,1,0,1])
    beta2 = np.array([-1,-1,-1,0,-1])
    p = len(beta1)
    Sigma = np.zeros((p,p));
    for i in range(p):
        for j in range(p):
            Sigma[i,j] = rho**(abs(i-j))
    X = np.random.multivariate_normal(mean =np.zeros(p), cov = Sigma, size = n)
    noise1 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta1)/SNR), size = int(n/2)) 
    noise2 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta2)/SNR), size = int(n/2)) 
    Y1 = X[:int(n/2)]@beta1 + noise1
    Y2 = X[int(n/2):]@beta2 + noise2
    Y = np.r_[Y1,Y2]
    lam = 0.1*n
    ols = sm.OLS(Y, X)
    ols_result = ols.fit()
    bigM = 10*float(1.96*(sum(ols_result.bse)))#1000

    M = grb.Model()
    M.Params.TimeLimit = 400
    beta = M.addMVar((int(n),5), lb = -grb.GRB.INFINITY)
    z = M.addMVar(len(Y) - 1, vtype = grb.GRB.BINARY)
    sum_of_sq = sum((Y[t] - sum(X[t,j]*beta[t,j] for j in range(5)))* (Y[t] - sum(X[t,j]*beta[t,j] for j in range(5))) for t in range(len(Y)))
    for t in range(len(Y)-1):
        for j in range(5):
            M.addConstr(beta[t+1,j] - beta[t,j] <= bigM*z[t])
            M.addConstr(beta[t+1,j] - beta[t,j] >= -bigM*z[t])
    #optional constratins to add more cuts
    M.addConstr(sum(z) <= 5)
    for t in range(1, len(Y) - 2):
        M.addConstr(z[t-1] + z[t] + z[t+1] <= 1)
    # quadratic penalty
    M.setObjective(sum_of_sq+ lam*sum(z)*sum(z))
    M.optimize()
    beta_hat = [beta[t].x for t in range(len(Y))]
    z_hat = [z[t].x for t in range(len(Y)-1)]
    opt_gap = M.MIPGap
    return z_hat, opt_gap,beta_hat

SNR_list = np.array([6, 3.52, 2.07, 1.22, 0.71])
rho_list = np.array([0, 0.3, 0.7])
n_list = np.array([1000])

header = ['Repitition','Time','rho', 'SNR', 'n', 'Optimality Gap', 'Change Points']#, 'beta_hat', 'nonzero count']
f = open('simulation_five_variate_quadratic_penalty.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)
f.close()

def to_repeat(rep):
    for n in n_list:
        for SNR in SNR_list:
            for rho in rho_list:
                tik = time.time()
                z_hat, opt_gap, beta_hat = five_variate(n,SNR,rho)
                tok = time.time()
                duration = tok - tik
                cps = [idx for idx,item in enumerate(z_hat) if item == 1 ]
                # Write result
                resultrow = [rep,duration,rho,SNR, n, opt_gap, cps]
                f = open('simulation_five_variate_quadratic_penalty.csv', 'a')
                writer = csv.writer(f)
                writer.writerow(resultrow)
                f.close()
                
Parallel(n_jobs = 8)(delayed(to_repeat)(rep) for rep in range(500)) #Tomo's machine has 8 cores.

Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter TimeLimit to value 400
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter TimeLimit to value 400
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter Username
Set parameter TimeLimit to value 400
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter TimeLimit to value 400
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter TimeLimit to value 400
Academic license - for non-commercial use only - expires 2024-02-18
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter TimeLimit to value 400
Set parameter TimeLimit to value 400
Academic license - for non-commercial use only - expires 2024-02-18
Set parameter


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    9699    2.4479794e+03   0.000000e+00   4.817701e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    9114    2.8036188e+03   0.000000e+00   5.287715e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8995    2.5247790e+03   0.000000e+00   5.350845e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8421    2.8083715e+03   0.000000e+00   6.551663e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8213    2.7869725e+03   0.000000e+00   6.230430e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    7311    2.9899079e+03   0.000000e+00   7.670485e+03      5s
   15503    2.1246186e+03   0.000000e+00   4.429128e+03     10s
   14155    1.9992195e+03   0.000000

     3     8  615.31344    2   89 4658.31820  615.31344  86.8%   926   73s
     1     4  621.88520    1   94 4934.00707  621.88520  87.4%  67.0   73s
     7    12  642.65187    3   75 4634.09428  642.65187  86.1%  1054   75s
     3     8  598.15533    2  110 4921.17444  598.15533  87.8%   947   77s
     7    12  721.29920    3   92 4836.83478  714.56807  85.2%  1206   77s
     3     8  691.17162    2   86 4794.12598  691.17162  85.6%   893   76s
     7    12  677.80549    3   90 4540.24059  677.80549  85.1%  1276   78s
     7    12  615.95966    3   90 4658.31820  615.31344  86.8%  1186   79s
     3     8  595.26570    2   83 4544.77988  595.26570  86.9%  1007   77s
    15    20  648.68341    5   73 4634.09428  642.79091  86.1%   530   81s
    11    16  726.11403    4   91 4836.83478  714.56807  85.2%   802   81s
    11    16  685.35565    4   89 4540.24059  677.96182  85.1%   840   82s
    11    16  616.38122    4   91 4658.31820  615.31344  86.8%   769   82s
     7    12  598.99244  

    75    39  669.64414   14   73  799.08591  642.79091  19.6%   197  159s
   276   127  747.59215   67   28  752.35205  616.57343  18.0%  98.3  161s
   190    25     cutoff   23       811.67760  734.46497  9.51%   145  161s
    88    31  739.76441   16   64  860.42433  714.56807  17.0%   197  161s
   107    67  612.84419   16   69  741.50558  599.31366  19.2%   158  163s
   100     4     cutoff    4       815.14330  722.14426  11.4%   181  161s
   165    79  659.27724   27   59  733.58601  634.44427  13.5%   115  161s
   299   130  748.07393   74   27  752.35205  616.57343  18.0%  94.8  165s
   125    70  612.85474   18   70  741.50558  599.31366  19.2%   141  165s
    88    41  669.71471   15   69  799.08591  642.79091  19.6%   174  165s
   222    22  796.85474   71   34  811.67760  734.46497  9.51%   135  166s
   104    35  753.68535   19   68  860.42433  714.56807  17.0%   182  167s
    61    34  755.35372   18   46  770.60921  629.61090  18.3%   273  167s
   116     4     cutoff  

   838     3  751.85634  192    5  752.35205  751.78263  0.08%  69.7  232s

Explored 861 nodes (96220 simplex iterations) in 233.14 seconds (122.15 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 752.352 4658.32 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.523520463830e+02, best bound 7.523520463830e+02, gap 0.0000%
   460   222  786.64021   13   46  799.08591  650.42844  18.6%  79.9  232s
Set parameter TimeLimit to value 400
   391    11  744.90375  105   52  815.14330  744.29270  8.69%  90.1  231s
   497   248  787.76106   22   43  799.08591  650.42844  18.6%  77.6  235s
   492   153  765.87595   20   42  770.60921  632.90081  17.9%  80.4  236s
   746    13     cutoff  196       860.42433  852.76987  0.89%  65.8  236s
   546   246  722.72060   34   43  733.58601  643.23085  12.3%  78.0  234s
   641   385  731.01566   73   47  741.42681  613.00684  17.3%  70.2  238s
   431     7     cutoff  111       815.14330  746.08777  8.47%  85.7  23

...

    41    40 4767.62132   11  150 5628.88787 4758.77660  15.5%   128   56s
   24712    4.7381547e+03   0.000000e+00   0.000000e+00     10s
   24730    4.7392261e+03   0.000000e+00   0.000000e+00     10s

Root relaxation: objective 4.739226e+03, 24730 iterations, 9.90 seconds (9.34 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4739.22606    0  135 9758.36190 4739.22606  51.4%     -   10s
    53    48 4770.26594   13  149 5628.88787 4758.77660  15.5%   106   61s
     0     0 4739.77307    0  135 9758.36190 4739.77307  51.4%     -   13s
    71    59 4781.72815   15  149 5628.88787 4758.77660  15.5%  87.6   65s
    90    69 4791.68938   17  151 5628.88787 4758.77660  15.5%  78.1   70s
   140    99 4837.41425   23  149 5628.88787 4758.77660  15.5%  62.2   76s
     0     2 4739.77307    0  135 9758.36190 4739.77307  51.4%     -   28s
   170   120 4874.98498   26 

  3254  3059 5876.73975  707   97 6076.33648 4758.95072  21.7%  28.3  322s
  4232  3548 4769.26131   12  133 5628.88787 4769.26131  15.3%  35.0  374s
  3487  3273 5937.39665  778   79 6076.33648 4758.95072  21.7%  28.1  330s
  4234  3551 4769.26131   13  140 5628.88787 4769.26131  15.3%  35.3  382s
  4238  3552 4769.26131   14  142 5628.88787 4769.26131  15.3%  35.7  388s
  3746  3400 5994.03737  849   64 6076.33648 4777.83927  21.4%  28.5  342s
H 3885  2572                    5737.2994950 4777.83927  16.7%  29.1  342s
  4242  3551 5149.45788   14  112 5628.88787 4769.26131  15.3%  36.1  396s
  4246  3551 4769.26131   15  145 5628.88787 4769.26131  15.3%  36.8  400s

Explored 4250 nodes (182223 simplex iterations) in 400.06 seconds (425.37 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 5628.89 9385.36 

Time limit reached
Best objective 5.628887873698e+03, best bound 4.769261305988e+03, gap 15.2717%
Set parameter TimeLimit to value 400
  3923  2635 5211.4

   101    95 6067.96341   19  176 7595.88691 6046.19869  20.4%  65.3  110s
   710   606 6130.79250  128  206 7359.24015 5779.07801  21.5%  33.3  171s
   132   124 6078.18924   22  177 7595.88691 6046.19869  20.4%  56.3  116s
   761   663 6162.24179  135  200 7359.24015 5779.07801  21.5%  32.6  176s
   173   161 6096.64522   34  185 7595.88691 6046.19869  20.4%  49.8  122s
   828   706 6234.63501  145  195 7359.24015 5779.07801  21.5%  31.9  180s
   194   181 6105.35967   39  185 7595.88691 6046.19869  20.4%  46.7  127s
   882   760 6415.75797  150  195 7359.24015 5779.07801  21.5%  31.8  186s
   217   204 6112.00035   43  188 7595.88691 6046.19869  20.4%  45.2  131s
   949   814 6454.32356  173  194 7359.24015 5779.07801  21.5%  31.5  191s
   259   238 6125.56459   55  183 7595.88691 6046.19869  20.4%  42.9  136s
  1016   885 6469.18547  191  212 7359.24015 5779.07801  21.5%  30.9  196s
   299   275 6128.90934   58  186 7595.88691 6046.19869  20.4%  39.7  141s
  1101   943 6486.55666  

     0     2 9808.06916    0  280 27193.0096 9808.06916  63.9%     -   43s
     1     4 9854.14437    1  282 27193.0096 9854.14437  63.8%   142   47s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   13269    1.1050703e+04   0.000000e+00   3.396717e+03      5s
     3     8 9871.92700    2  281 27193.0096 9871.92700  63.7%   231   51s
   18131    9.9711136e+03   0.000000e+00   1.267689e+03     11s
     7    12 9892.85596    3  282 27193.0096 9877.10732  63.7%   273   56s
   21055    9.5390661e+03   0.000000e+00   3.498278e+02     15s
   22739    9.4246522e+03   0.000000e+00   0.000000e+00     19s

Root relaxation: objective 9.424652e+03, 22739 iterations, 19.03 seconds (14.93 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9424.65222    0  278 23885.5320 9424.65222  60.5%     -   19s
    11    16 9903.14048    4  283 2719

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]